In [21]:
import os
from urllib.parse import urlparse, parse_qs

# Đường dẫn đến thư mục chứa các file __label__
folder_path = 'label'

# Lấy danh sách tất cả các tệp và thư mục trong thư mục
contentLabels = os.listdir(folder_path)

In [2]:
with open('assign_labels.txt', 'a+', encoding='utf-8') as file:
    labelLines = file.readlines()
    for contentLabel in contentLabels:
        contentLabelWasReplaced = contentLabel.replace(contentLabel[-4:], '')
        if (contentLabelWasReplaced + '\n' not in labelLines):
            file.write(contentLabelWasReplaced + '\n')

In [3]:
with open('assign_labels.txt', 'r', encoding='utf-8') as file:
    labelLines = file.readlines()

with open('assign_labels.txt', 'w', encoding='utf-8') as file:
    for contentLabel in contentLabels:
        contentLabelWasReplaced = contentLabel.replace(contentLabel[-4:], '')
        with open(folder_path + '/' + contentLabel, 'r', encoding='utf-8') as fileContentLabel:
            contentLabelLines = fileContentLabel.readlines()
            previous_domain = ''
            last_line = ''
            for labelLine in labelLines:
                for contentLabelLine in contentLabelLines:
                    url = contentLabelLine.strip().split(' ', 1)
                    parsed_url = urlparse(url[0])
                    domain = parsed_url.netloc
                    if (previous_domain != domain):
                        previous_domain = domain
                        split_labelLine_result = labelLine.strip().split(' ', 1)
                        if len(split_labelLine_result) == 2:
                            label, keyword = split_labelLine_result
                            if (domain not in keyword and label == contentLabelWasReplaced):
                                labelLine = labelLine.replace(keyword, keyword + ' ' + domain)
                            last_line = labelLine
                        else:
                            label = labelLine.strip()
                            if (label == contentLabelWasReplaced):
                                labelLine = labelLine.strip() + ' ' + domain
            file.write(last_line + '\n')

In [22]:
import requests
from bs4 import BeautifulSoup
from bs4.element import Comment
from urllib.parse import urlparse, parse_qs

#Xác định hàm tag_visible để lọc các thẻ không mong muốn trong HTML
def tag_visible(element):
    blacklist = ['style', 'label', '[document]', 'embed'
                 'noscript', 'header', 'html', 'iframe'
                 'meta', 'title', 'aside', 'footer',
                 'form', 'nav', 'head', 'link', 
                 'br', 'input', 'script', 'figure']
    if element.parent.name in blacklist:
        return False
    if isinstance(element, Comment):
        return False
    return True

def get_page_content(url):
    try:
        response = requests.get(url)
        response.raise_for_status()
        return response.text
    except requests.exceptions.RequestException as e:
        # print('Đã xảy ra lỗi:', e)
        return None

def extract_content(html_content, labelAssigned):
    if html_content is None:
        return None, None
    soup = BeautifulSoup(html_content, 'lxml')
    
    title = soup.title.text.strip()
    paragraphs = soup.body.findAll(string=True)
    visible_texts = filter(tag_visible, paragraphs)
    cleaned_texts = (t.strip() for t in visible_texts if t.strip())  # Loại bỏ các chuỗi rỗng và loại bỏ khoảng trắng
    content = labelAssigned + ' ' + ' '.join(cleaned_texts)
    # Lấy 5 dòng đầu tiên
    # lines = content.split('\n')
    # content = labelAssigned + ' ' + '\n'.join(lines[:15])
    return title, content


### <b>Hàm để trích xuất các từ khóa từ URL</b>

In [23]:
def extract_keywords_from_url(url):
    parsed_url = urlparse(url)
    path_keywords = parsed_url.path.split('/')
    query_params = parsed_url.query.split('&')
    keywords = path_keywords + query_params
    return [keyword for keyword in keywords if '=' not in keyword]  # Loại bỏ các tham số truy vấn

### <b>Đọc các URL từ tệp văn bản</b>

In [24]:
def read_urls_from_file(file_path):
    with open(file_path, 'r') as file:
        urls = [line.strip() for line in file.readlines()]
        print(urls)
    return urls

### <b>Ghi nội dung vào tệp văn bản</b>

In [25]:
def write_content_to_file(file_path, title, content):
    with open(file_path, 'a', encoding='utf-8') as file:
        file.write('Title: ' + title + '\n')
        file.write('Content: ' + content + '\n\n')

### <b>Hàm gán nhãn</b>

In [26]:
# Đọc các nhãn từ file và tạo từ điển label_mapping
label_mapping = {}
with open('assign_labels.txt', 'r', encoding='utf-8') as file:
    for line in file:
        parts = line.strip().split()
        label = parts[0]
        keywords = parts[1:]
        label_mapping[label] = keywords

# Hàm để gán nhãn cho một đoạn văn
def assign_label(text):
    for label, keywords in label_mapping.items():
        for keyword in keywords:
            if keyword in text:
                return label
    return None

In [27]:
def clean_data(data):
    # Loại bỏ ký tự xuống dòng và khoảng trắng thừa
    cleaned_data = ' '.join(data.split())
    return cleaned_data

### <b>Duyệt qua từng URL và lấy nội dung</b>

In [28]:
for contentLabel in contentLabels:
    with open(folder_path + '/' + contentLabel, 'r', encoding='utf-8') as fileContainUrl:
        contentLabelLines = fileContainUrl.readlines()
        contentLabelWasReplaced = contentLabel.replace(contentLabel[-4:], '')
        for contentLabelLine in contentLabelLines:
            url = contentLabelLine.strip().split(' ', 1)[0]
            # print(domain)
            html_content = get_page_content(url)
            if html_content:
                # Trích xuất tiêu đề và nội dung từ HTML
                try:
                    title, content = extract_content(html_content, assign_label(url))
                    content = clean_data(content)
                except:
                    print('except:', url)
                    continue                
                # Ghi tiêu đề và nội dung vào tệp văn bản
                output_directory = 'output'
                if not os.path.exists(output_directory):
                    os.makedirs(output_directory)
                output_file = os.path.join(output_directory, contentLabelWasReplaced + '_output' + '.txt')
                write_content_to_file(output_file, title, content)
            else:
                print('Không thể lấy nội dung từ:', url)

Không thể lấy nội dung từ: https://www.webtretho.com/
Không thể lấy nội dung từ: https://www.webtretho.com/thinh-hanh
Không thể lấy nội dung từ: https://www.webtretho.com/c/tat-ca
Không thể lấy nội dung từ: https://www.webtretho.com/f/mang-thai-ba-bau
Không thể lấy nội dung từ: https://www.webtretho.com/f/mang-thai-ba-bau/lop-tien-san-mien-phi-tai-benh-vien-phu-san-quoc-te-sai-gon
Không thể lấy nội dung từ: https://www.webtretho.com/f/mang-thai-ba-bau/lop-tien-san-mien-phi-tai-benh-vien-phu-san-quoc-te-sai-gon
Không thể lấy nội dung từ: https://www.webtretho.com/f/cafe-webtrethonofollow
Không thể lấy nội dung từ: https://www.webtretho.com/f/cafe-webtretho
Không thể lấy nội dung từ: https://www.webtretho.com/f/cafe-webtretho/yokogold-dong-hanh-cung-ngay-hoi-webtretho-baby-fair-2024
Không thể lấy nội dung từ: https://www.webtretho.com/f/cafe-webtretho/yokogold-dong-hanh-cung-ngay-hoi-webtretho-baby-fair-2024
Không thể lấy nội dung từ: https://www.webtretho.com/f/vat-dung-nha-bep-thiet-bi